In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
VECTORSTORE_PATH = os.getenv("VECTORSTORE_PATH")

In [2]:
from langchain_community.vectorstores.faiss import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct

# OpenAI 임베딩 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)

# Qdrant 클라이언트 초기화
client = QdrantClient(" https://qdrant-service-348189284538.asia-northeast3.run.app")  # Qdrant 서버 주소에 맞게 수정


C:\Users\GFCON\AppData\Local\Temp\ipykernel_11504\731999492.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=OPENAI_API_KEY)
C:\Users\GFCON\AppData\Local\Temp\ipykernel_11504\731999492.py:10: UserWarning: Failed to obtain server version. Unable to check client-server compatibility. Set check_version=False to skip version check.
  client = QdrantClient(" https://qdrant-service-348189284538.asia-northeast3.run.app")  # Qdrant 서버 주소에 맞게 수정


In [3]:
https://qdrant-service-348189284538.asia-northeast3.run.app/print(os.getcwd())
store_type = "title"
vectorestore_dir = f"../../ilora-clone/rag_code/vectorstore/{store_type}_search_store"
print(os.listdir(vectorestore_dir))
# FAISS 인덱스 로드
faiss_store = FAISS.load_local(vectorestore_dir, embeddings, allow_dangerous_deserialization=True)

c:\Users\GFCON\Documents\Project\QdrantMigration\migrate_vectorstore
['index.faiss', 'index.pkl']


In [12]:
# Qdrant에 업로드할 포인트 데이터 변환
from tqdm import tqdm
points = []
for i in tqdm(range(len(faiss_store.index_to_docstore_id)), desc="Converting FAISS to Qdrant points", position=0):
    doc_id = faiss_store.index_to_docstore_id[i]
    doc = faiss_store.docstore.search(doc_id)
    vector = faiss_store.index.reconstruct(i)
    payload = doc.metadata
    point = PointStruct(id=doc_id, vector=vector, payload=payload)
    points.append(point)


Converting FAISS to Qdrant points:   0%|          | 0/195739 [00:00<?, ?it/s]

Converting FAISS to Qdrant points:  11%|█         | 21136/195739 [00:03<00:32, 5433.48it/s]


KeyboardInterrupt: 

In [12]:
# 새로운 컬렉션 생성
from qdrant_client.models import VectorParams, Distance, CollectionConfig
client = QdrantClient("https://qdrant-service-348189284538.asia-northeast3.run.app:6333")
collection_name = f"pubmed_{store_type}_search"

vector_size = 1536
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE),
)

C:\Users\GFCON\AppData\Local\Temp\ipykernel_10252\3673459793.py:7: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


ResponseHandlingException: timed out

In [38]:
pip install qdrant-client

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\GFCON\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [43]:
from qdrant_client import QdrantClient

client = QdrantClient(
    url="https://qdrant-service-348189284538.asia-northeast3.run.app",
    https=False,
    timeout=60
)

In [27]:
# upload points to qdrant by batch 500

batch_size = 200
for i in tqdm(range(0, len(points), batch_size), desc="Uploading points to Qdrant", position=0):
    batch_points = points[i:i+batch_size]
    client.upsert(collection_name=collection_name, points=batch_points)


Uploading points to Qdrant:   0%|          | 0/39148 [00:20<?, ?it/s]


ResponseHandlingException: timed out

In [32]:
from qdrant_client import QdrantClient
from qdrant_client.models import SearchParams
import numpy as np

# Cloud Run에 배포된 Qdrant 서비스 URL을 사용해 클라이언트 초기화
client = QdrantClient("http://localhost:6333")

# 검색할 컬렉션 이름 (이전에 생성한 컬렉션 이름으로 대체)
collection_name = "pubmed_abstract_search"

# 예시 query vector (차원 1536; 실제 사용 시에는 임베딩 모델을 통해 생성한 벡터를 사용하세요)
query_vector = np.random.rand(1536).tolist()

# Qdrant 검색 실행 (상위 10개 결과 반환, payload 포함)
results = client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=10,
    with_payload=True,
    
    #params=SearchParams(hnsw_ef=128)  # 필요에 따라 조정 가능
)

print("Search Results:")
for result in results:
    print(result)


C:\Users\GFCON\AppData\Local\Temp\ipykernel_10252\4197856698.py:15: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


Search Results:
id='68543f2b-e6f6-4267-ac14-1863983ca981' version=240 score=0.07299339 payload={'title': 'Cardiovascular Dysfunction in Intrauterine Growth Restriction.', 'authors': ['Amruta N', 'Kandikattu HK', 'Intapad S'], 'publication_date': '2022 Dec', 'journal': 'Current hypertension reports', 'doi': '', 'pmid': '36322299', 'source': 'Cardiovascular Dysfunction in Intrauterine Growth _36322299', 'abstract': 'PURPOSE OF REVIEW: We highlight important new findings on cardiovascular  dysfunction in intrauterine growth restriction. RECENT FINDINGS: Intrauterine  growth restriction (IUGR) is a multifactorial condition which negatively impacts  neonatal growth during pregnancy and is associated with health problems during  the lifespan. It affects 5-15% of all pregnancies in the USA and Europe with  varying percentages in developing countries. Epidemiological studies have  reported that IUGR is associated with the pathogenesis of hypertension,  activation of the renin-angiotensin syste

In [44]:
import requests

# Qdrant 서비스의 URL
qdrant_url = "https://qdrant-service-348189284538.asia-northeast3.run.app"

# '/collections' 엔드포인트에 GET 요청 보내기
response = requests.get(f"{qdrant_url}/collections")

# 응답 상태 코드 확인
if response.status_code == 200:
    # JSON 형식으로 응답 데이터 출력
    collections = response.json()
    print("Collections:", collections)
else:
    print(f"Failed to retrieve collections. Status code: {response.status_code}")


Collections: {'result': {'collections': [{'name': 'star_charts'}]}, 'status': 'ok', 'time': 1.9902e-05}


In [ ]:
# upload points to qdrant by batch 500

batch_size = 2
for i in tqdm(range(0, len(points), batch_size), desc="Uploading points to Qdrant", position=0):
    batch_points = points[i:i+batch_size]
    client.upsert(collection_name=collection_name, points=batch_points)


In [ ]:
import requests
import json

# Qdrant 서비스 URL 및 컬렉션 이름 설정
     # Qdrant 서비스의 URL로 변경하세요
collection_name = "pubmed_abstract_search"  # 업로드할 컬렉션 이름으로 변경하세요
# Specify the collection name
# collection_name = "your_collection_name"

# Define the collection configuration
collection_config = {
    "vectors": {
        "size": 1536,  # Dimensionality of your vectors
        "distance": "Cosine"  # Distance metric: "Cosine", "Euclidean", or "Dot"
    }
}

# Send a PUT request to create the collection
response = requests.put(
    f"{qdrant_url}/collections/{collection_name}",
    headers={"Content-Type": "application/json"},
    data=json.dumps(collection_config)
)
# 업로드할 포인트 데이터 정의


In [10]:
# REST API 엔드포인트 URL
# url = f"{qdrant_url}/collections/{collection_name}/points"
import requests
import json
from tqdm import tqdm
import time
# Qdrant에 업로드할 포인트 데이터 변환
from tqdm import tqdm
points = []
for i in tqdm(range(1000), desc="Converting FAISS to Qdrant points", position=0):
    doc_id = faiss_store.index_to_docstore_id[i]
    doc = faiss_store.docstore.search(doc_id)
    vector = faiss_store.index.reconstruct(i)
    payload = doc.metadata
    point = PointStruct(id=doc_id, vector=vector, payload=payload)
    points.append(point)

url = "https://qdrant-service-348189284538.asia-northeast3.run.app"
# 요청 본문 생성
# batch size 3
batch_size = 3
for i in tqdm(range(0, len(points), batch_size), desc="Uploading points to Qdrant", position=0):
    # to dictionary
    batch_points = [point.model_dump() for point in points[i:i+batch_size]]
    # batch_points = points[i:i+batch_size]
    # payload = {
    #     "points": batch_points
    # }

    # HTTP PUT 요청 보내기
    response = requests.put(url, headers={"Content-Type": "application/json"}, data=json.dumps(batch_points))

    # 응답 확인
    if response.status_code == 200:
        print("포인트 업로드 성공")
    else:
        print(f"포인트 업로드 실패: {response.status_code}")
        print(response.json())


Uploading points to Qdrant:   0%|          | 0/334 [00:00<?, ?it/s]

포인트 업로드 실패: 404


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
import requests
import json
from tqdm import tqdm

# Qdrant 서비스 URL 및 컬렉션 이름 설정
qdrant_url = "https://qdrant-service-348189284538.asia-northeast3.run.app"
import requests
import json
from tqdm import tqdm
import time
from pydantic import BaseModel
from typing import List, Dict, Any

# Qdrant Point 구조 정의
class PointStruct(BaseModel):
    id: str
    vector: List[float]
    payload: Dict[str, Any]

# Qdrant 업로드를 위한 데이터 변환
points = []
for i in tqdm(range(len(faiss_store.index_to_docstore_id)), desc="Converting FAISS to Qdrant points", position=0):
    doc_id = faiss_store.index_to_docstore_id[i]
    doc = faiss_store.docstore.search(doc_id)
    vector = faiss_store.index.reconstruct(i)
    payload = doc.metadata
    point = PointStruct(id=doc_id, vector=vector, payload=payload)
    points.append(point)

# Qdrant API URL 설정
qdrant_url = "https://qdrant-service-348189284538.asia-northeast3.run.app"
collection_name = "pubmed_abstract_search"   # 실제 컬렉션 이름으로 변경
url = f"{qdrant_url}/collections/{collection_name}/points"

# 요청 본문 생성 및 업로드
batch_size = 300
headers = {"Content-Type": "application/json"}

for i in tqdm(range(0, len(points), batch_size), desc="Uploading points to Qdrant", position=0):
    batch_points = [point.model_dump() for point in points[i:i+batch_size]]
    payload = {"points": batch_points}
    
    try:
        response = requests.put(url, headers=headers, json=payload)
    except requests.exceptions.RequestException as e:
        print(f"요청 중 오류 발생: {e}")


Uploading points to Qdrant:  35%|███▌      | 230/653 [19:33<37:07,  5.27s/it]  